<a href="https://colab.research.google.com/github/Taeseong-eom/ApplePython/blob/main/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

print(data.isnull().sum())
# Age 칼럼에 빈칸이 177개라서  평균값을 넣었다.
# 평균 = data['Age'].mean()
# print(평균) # 29.69911764705882
data['Age'].fillna(value=30, inplace=True) # 평균을 반올림 하여 적용하였다.

# Embarked 칼럼도 2개가 최빈값으로 채우겠다.
# 최빈값 = data['Embarked'].mode()
# print(최빈값)
data['Embarked'].fillna(value='S', inplace=True)

print(data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import tensorflow as tf

# tf.data.Dataset.from_tensor_slices(X데이터, 정답) # 텐서플로우 데이터셋에 맞게 만들어줌
# X 데이터는 정답을 제외한 모든 데이터고 정답데이터는 Survived 임.
# 데이터를 그럼 분리를 해보자
정답 = data.pop('Survived')
train_size = int(0.8 * len(data))
train_ds = tf.data.Dataset.from_tensor_slices((dict(data[:train_size]), 정답[:train_size]))
val_ds = tf.data.Dataset.from_tensor_slices((dict(data[train_size:]), 정답[train_size:]))

# 지금까지는 [] 데이터만 학습데이터로 넣었는데 딕셔너리도 넣을 수 있다.
print(train_ds)

<_TensorSliceDataset element_spec=({'PassengerId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [15]:
import tensorflow as tf

# feature_column을 쓰려면 위에서 만든 ds처럼 데이터셋을 만들어야한다.
# 필요 없는건 버리겠음. PassengerId	Name

# 그대로 집어넣을꺼 SibSp	Parch Fare : numeric_column

# 묶어서 넣을꺼 Age : bucketized_column

# 카테고리/원핫인코딩할꺼(종류 몇개 없는거) Sex Embarked  : indicator_columnPclass

# 종류가 많은거 Ticket : embedding_column # 하나의 행렬로 표현하고 무작위 숫자부여-> 학습할수록 값 조정

feature_columns=[]
feature_columns.append( tf.feature_column.numeric_column('Fare') )
feature_columns.append( tf.feature_column.numeric_column('Parch'))
feature_columns.append( tf.feature_column.numeric_column('SibSp'))

Age = tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries=[10,20,30,40,50,60]) # 10, 20, 30 대로 묶어줌.
feature_columns.append( Age_bucket )

print(Age)
print(feature_columns)

NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
[NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(10, 20, 30, 40, 50, 60))]


In [16]:
vocab = data['Sex'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab) # 카테고리 나눌건데 뒤에 유니크한 리스트를 넣어야함.
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)

vocab = data['Embarked'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab) # 카테고리 나눌건데 뒤에 유니크한 리스트를 넣어야함.
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)

vocab = data['Pclass'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab) # 카테고리 나눌건데 뒤에 유니크한 리스트를 넣어야함.
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)


#embedding
vocab = data['Ticket'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab) # 카테고리 나눌건데 뒤에 유니크한 리스트를 넣어야함.
one_hot = tf.feature_column.embedding_column(cat, dimension=9)
feature_columns.append(one_hot)

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns=feature_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation='sigmoid'), # 0 과 1 사이 출력
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

ds_batch =  train_ds.batch(32) # 32개씩 배치해줌.
val_ds_batch = val_ds.batch(32)

model.fit(ds_batch,validation_data=val_ds_batch ,shuffle=True, epochs=20)

Epoch 1/20
23/23 [==============================] - 2s 19ms/step - loss: 1.2258 - acc: 0.5449 - val_loss: 0.7912 - val_acc: 0.6480
Epoch 2/20
23/23 [==============================] - 0s 5ms/step - loss: 0.8570 - acc: 0.6334 - val_loss: 0.5157 - val_acc: 0.8156
Epoch 3/20
23/23 [==============================] - 0s 4ms/step - loss: 0.7354 - acc: 0.6980 - val_loss: 0.4758 - val_acc: 0.7933
Epoch 4/20
23/23 [==============================] - 0s 4ms/step - loss: 0.7399 - acc: 0.7191 - val_loss: 0.4320 - val_acc: 0.8101
Epoch 5/20
23/23 [==============================] - 0s 4ms/step - loss: 0.5846 - acc: 0.7598 - val_loss: 0.4322 - val_acc: 0.8156
Epoch 6/20
23/23 [==============================] - 0s 4ms/step - loss: 0.5676 - acc: 0.7697 - val_loss: 0.4142 - val_acc: 0.8212
Epoch 7/20
23/23 [==============================] - 0s 4ms/step - loss: 0.5239 - acc: 0.7935 - val_loss: 0.4035 - val_acc: 0.8324
Epoch 8/20
23/23 [==============================] - 0s 4ms/step - loss: 0.5469 - acc: 0.7